In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from unidecode import unidecode
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
#FatosGeração (ID empreendiemnto) > fatodadosmetereologicos (vento, precip, etc) > dimempreendimentos (tipogeração)

#dim_empreendimentos -> id;NomEmpreendimento;SigUFPrincipal;SigTipoGeracao;NomFonteCombustivel
#fato_dados_metereologicos -> id;Data;Hora UTC;precipTotalHorario;ventoVeloHoraria;tempAr;radGlobal;umidRelAr;id_regiao
#fato_geracao -> id;DatGeracaoConjuntoDados;MdaPotenciaFiscalizadaKw;MdaPotenciaOutorgadaKw;Id_empreendimento
#id;UF;latitude;longitude;altitude
#Pegar empreendimento (id) > Pegar todos os dados meterologicos (preciptação, vento, temp...) > Pegar qual o TipoGeração na região

dim_regiao = pd.read_csv(
    "data/dim_regiao.csv",
    delimiter=";",
    encoding="UTF-8")

dim_empreendimentos = pd.read_csv(
    "data/dim_empreendimento.csv",
    delimiter=";",
    encoding="UTF-8")

fato_dados_metereologicos = pd.read_csv(
    "data/fato_dados_meteorologicos.csv",
    delimiter=";",
    encoding="UTF-8")

fato_geracao = pd.read_csv(
    "data/fato_geracao.csv",
    delimiter=";",
    encoding="UTF-8",
    decimal=',')

#sklearn.ensemble.HistGradientBoostingClassifier

In [4]:
# dim_empreendimentos['DscMuninicpios'].str.split('-')[0][0].strip()

dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].apply(lambda x: unidecode(str(x).split('-')[0].strip().upper()))
dim_empreendimentos['DscMuninicpios']

0               NOVA LIMA
1               NOVA LIMA
2               NOVA LIMA
3         BENTO GONCALVES
4        FLORIANO PEIXOTO
               ...       
24595              MANAUS
24596              MANAUS
24597              GOIANA
24598             GOIOERE
24599           CARIACICA
Name: DscMuninicpios, Length: 24600, dtype: object

In [5]:
#dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].str.split('-', 1).str[0]
# dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].str.split('-')[0][0].strip()
# dim_empreendimentos['DscMuninicpios'] = dim_empreendimentos['DscMuninicpios'].str.upper()
empreendimentos_combinados = pd.merge(fato_geracao, dim_empreendimentos, left_on='Id_empreendimento', right_on='id')
#dim_empreendimentos.head()







empreendimentos_combinados = empreendimentos_combinados.drop('DatGeracaoConjuntoDados', axis=1)

empreendimentos_combinados.head()

,id_x,MdaPotenciaFiscalizadaKw,MdaPotenciaOutorgadaKw,Id_empreendimento,id_y,NomEmpreendimento,SigUFPrincipal,SigTipoGeracao,NomFonteCombustivel,DscMuninicpios
0,0,1400,1400.0,0,0,E,MG,PCH,Potencial hidráulico,NOVA LIMA
1,1,3972,3972.0,1,1,F,MG,PCH,Potencial hidráulico,NOVA LIMA
2,2,1440,1440.0,2,2,G,MG,PCH,Potencial hidráulico,NOVA LIMA
3,3,100000,100000.0,3,3,14 de Julho,RS,UHE,Potencial hidráulico,BENTO GONCALVES
4,4,720,720.0,4,4,Abaúna,RS,CGH,Potencial hidráulico,FLORIANO PEIXOTO


In [6]:
dim_regiao.head()

,id,UF,cidade,latitude,longitude,altitude
0,0,DF,BRASILIA,"-15,78944444","-47,92583332","1160,96"
1,1,DF,BRAZLANDIA,"-15,59972221","-48,1311111",1143
2,2,DF,AGUAS EMENDADAS,"-15,596491","-47,625801","1030,36"
3,3,DF,GAMA (PONTE ALTA),"-15,93527777","-48,13749999",990
4,4,DF,PARANOA (COOPA-DF),"-16,012222","-47,557417",1043


In [7]:
#Cidade, MediaPrecipTotalHorarioCidade, MediaVentoVeloHorariaCidade, MediaTempArCidade, ...., QuantidadeEnergiaEnergiaSolarGeradaPorCidade, QuantidadeEnergiaNuclearGeradaPorCidade, QuantidadeEnergiaTermalGeradaPorCidade, .....

dados_combinados = pd.merge(fato_dados_metereologicos, dim_regiao[['id', 'UF','cidade']], left_on='id_regiao', right_on='id')

#Tratando dados decimais para virarem tipo float
#dados_combinados = pd.merge(fato_dados_metereologicos, dim_regiao, left_on='id_regiao', right_on='id')

dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].astype(str)
dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].str.replace(',', '.')
dados_combinados['precipTotalHorario'] = dados_combinados['precipTotalHorario'].astype(float)

dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].astype(str)
dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].str.replace(',', '.')
dados_combinados['ventoVeloHoraria'] = dados_combinados['ventoVeloHoraria'].astype(float)

dados_combinados['tempAr'] = dados_combinados['tempAr'].astype(str)
dados_combinados['tempAr'] = dados_combinados['tempAr'].str.replace(',', '.')
dados_combinados['tempAr'] = dados_combinados['tempAr'].astype(float)

dados_combinados['radGlobal'] = dados_combinados['radGlobal'].astype(str)
dados_combinados['radGlobal'] = dados_combinados['radGlobal'].str.replace(',', '.')
dados_combinados['radGlobal'] = dados_combinados['radGlobal'].astype(float)

In [8]:
siglasUF = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']


#dados_combinados.head()

#dados_combinados['cidade']
sample_dados_combinados = dados_combinados.sample(frac=0.1, random_state=42)
sample_dados_combinados

,id_x,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao,timestamp,id_y,UF,cidade
915995,915995,0.0,0.9,27.700000,1509.940151,59.0,169,2022-07-16 23:00:00,169,MA,CHAPADINHA
859452,859452,0.0,1.8,24.100000,1963.832282,80.0,156,2022-12-19 06:00:00,156,CE,QUIXERAMOBIM
1804653,1804653,0.0,0.0,21.400000,1262.124949,59.0,348,2022-09-10 23:00:00,348,MG,VICOSA
1908538,1908538,0.0,0.2,19.500000,1414.209515,98.0,361,2022-01-08 07:00:00,361,MG,PATROCINIO
3163682,3163682,0.0,3.6,22.600000,1965.300000,75.0,526,2022-10-06 18:00:00,526,RS,FREDERICO WESTPHALEN
...,...,...,...,...,...,...,...,...,...,...,...
3068208,3068208,0.0,1.4,17.700000,1375.567383,76.0,514,2022-12-29 07:00:00,514,RS,QUARAI
2624005,2624005,0.0,1.5,19.900000,1637.200000,68.0,450,2022-11-18 11:00:00,450,SP,TAUBATE
1404208,1404208,0.0,0.0,13.414706,0.000000,92.0,278,2022-05-15 13:00:00,278,PA,RONDON DO PARA
1289597,1289597,0.0,0.9,28.500000,856.000000,68.0,255,2022-08-12 12:00:00,255,AM,MANACAPURU


In [9]:
df_final = pd.merge(sample_dados_combinados, empreendimentos_combinados, left_on='cidade', right_on='DscMuninicpios')


df_final

,id_x_x,precipTotalHorario,ventoVeloHoraria,tempAr,radGlobal,umidRelAr,id_regiao,timestamp,id_y_x,UF,...,id_x_y,MdaPotenciaFiscalizadaKw,MdaPotenciaOutorgadaKw,Id_empreendimento,id_y_y,NomEmpreendimento,SigUFPrincipal,SigTipoGeracao,NomFonteCombustivel,DscMuninicpios
0,1908538,0.0,0.2,19.5,1414.209515,98.0,361,2022-01-08 07:00:00,361,MG,...,1993,186,186.0,1993,1993,Distribuidora de Carnes Patrocínio Agropecuária,MG,UTE,Óleo Diesel,PATROCINIO
1,1908538,0.0,0.2,19.5,1414.209515,98.0,361,2022-01-08 07:00:00,361,MG,...,2612,80,80.0,2612,2612,Granja Makena,MG,UTE,Biogás - RA,PATROCINIO
2,1908538,0.0,0.2,19.5,1414.209515,98.0,361,2022-01-08 07:00:00,361,MG,...,2903,300,300.0,2903,2903,Parque Hotel Serra Negra,MG,CGH,Potencial hidráulico,PATROCINIO
3,1908538,0.0,0.2,19.5,1414.209515,98.0,361,2022-01-08 07:00:00,361,MG,...,3045,265,265.0,3045,3045,Supermercado Bernardão,MG,UTE,Óleo Diesel,PATROCINIO
4,1908538,0.0,0.2,19.5,1414.209515,98.0,361,2022-01-08 07:00:00,361,MG,...,3073,347,347.0,3073,3073,Hospital e Maternidade Med Center,MG,UTE,Óleo Diesel,PATROCINIO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536822,1430435,0.0,1.5,28.2,1233.300000,70.0,283,2022-04-12 20:00:00,283,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES
4536823,1430493,0.0,0.2,24.3,0.000000,93.0,283,2022-04-15 06:00:00,283,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES
4536824,1430439,0.0,0.5,24.2,0.000000,91.0,283,2022-04-13 00:00:00,283,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES
4536825,1430386,0.0,2.2,30.1,2061.800000,65.0,283,2022-04-10 19:00:00,283,PA,...,2407,60,60.0,2407,2407,Santo Antônio,PA,UTE,Resíduos Florestais,BREVES


In [10]:
colunas_com_nan = df_final.columns[df_final.isnull().any()].tolist()
print("Colunas com valores NaN:")
for coluna in colunas_com_nan:
    print(coluna)

Colunas com valores NaN:


In [12]:
X = df_final[['MdaPotenciaOutorgadaKw', 'precipTotalHorario', 'ventoVeloHoraria', 'tempAr', 'radGlobal', 'umidRelAr']]
y = df_final['SigTipoGeracao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


clf = DecisionTreeClassifier()


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Precisão:", accuracy)

Precisão: 0.9863781539092273
